In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits  # 손글씨 숫자 데이터 (0~9)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score
)

# 한글 폰트 및 그래프 스타일 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PyTorch 버전: {torch.__version__}")
print(f"NumPy 버전: {np.__version__}")

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def evaluate_model(model, inputs, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        predicted = torch.max(outputs, 1)[1]

        correct = (predicted == labels).sum().item()
        acc = correct / len(labels)

    return loss.item(), acc

In [ ]:
def train_model(model, train_features, train_labels, val_features, val_labels, num_epochs=200,print_interval=20):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    print(f"{'Epoch':^10} | {'Train Loss':^12} | {'Train Acc':^10} | {'Val Loss':^12} | {'Val Acc':^10}")
    print('='*70)
    for epoch in range(num_epochs):
        model.train()
        outputs = model(train_features)
        loss = criterion(outputs, train_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 트레인
        train_loss, train_acc = evaluate_model(model, train_features, train_labels)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        # val
        val_loss, val_acc = evaluate_model(model, val_features, val_labels)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        if (epoch + 1) % print_interval == 0:
            print(f'{epoch + 1:^10} | '
                  f'{train_loss:^12.4f} | '
                  f'{train_acc:^10.4f} | '
                  f'{val_loss:^12.4f} | '
                  f'{val_acc:^10.4f}'
                  )
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }

In [ ]:
def plot_training_history(history):
    """
    학습 과정을 시각화하는 함수


    Args:
        history: 학습 이력 딕셔너리
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))


    # 손실(Loss) 그래프
    axes[0].plot(history['train_losses'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_losses'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Loss Curve', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)


    # 정확도(Accuracy) 그래프
    axes[1].plot(history['train_accs'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history['val_accs'], label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title('Accuracy Curve', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


    # 최종 결과 출력
    print("\n" + "=" * 50)
    print("최종 학습 결과")
    print("=" * 50)
    print(f"훈련 손실: {history['train_losses'][-1]:.4f}")
    print(f"검증 손실: {history['val_losses'][-1]:.4f}")
    print(f"훈련 정확도: {history['train_accs'][-1]:.4f} ({history['train_accs'][-1]*100:.2f}%)")
    print(f"검증 정확도: {history['val_accs'][-1]:.4f} ({history['val_accs'][-1]*100:.2f}%)")
    print("=" * 50)

